In [ ]:
#Importing the required libraries for building the nueral network
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import pickle
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#Defining Start and end of the sentense. SOS - start of the sentence and EOS - end of the sentences
SOS_token = 0
EOS_token = 1

#Creating word to index and frequency of each words in a dictionary. As the nueral network accepts input as a vector,
#We index every word with a number. Each unique number or vector could represent a word for input and output of text.
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
# Turn a Unicode string to plain ASCII

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
data=pd.read_csv('eng-kan-180k.csv')
data.head()
# # print(len(data['Go']))
eng=data['0']
print(len(eng))
kan=data['1']
# temp=[]
# temp=eng
# eng=kan
# kan=temp
a=[]
b=[]
for i,j in zip(eng,kan):
  try:
    if(len(i.split())>=15 or len(j.split())>=15 ):continue
    a.append(i)
    b.append(j);
  except:
    continue
eng=a
kan=b
X_train, X_test, y_train, y_test = train_test_split(eng, kan, test_size=0.3, random_state=42)
print(len(X_train))

def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines, 
    # the file in uploaded to the sessions cache of google colab, the file address needs to changed
    # lines = open('data.txt', encoding='utf-8').\
    #     read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = []
    l = []
    itr=0
    eng=X_train
    kan=y_train
#     print(eng[8],kan[8])
    for p,q in zip(X_train,y_train):
      if(len(p.split())>=15 or len(q.split())>=15 ):continue
      l.append([p,q])
      itr=itr+1
    pairs = l
#     print(pairs)
    print("---->",len(pairs))

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)
    val=pairs
    return input_lang, output_lang, pairs


199401
129817


In [ ]:
#Declaring function to format data into readable formats
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1,lang2,reverse)
    print("Read %s sentence pairs" % len(pairs))
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs




In [ ]:
#The following is a sample list, [Kannada, English]

input_lang_kan, output_lang_eng, pairs = prepareData('eng','kan',True)
print(random.choice(pairs))

Reading lines...
----> 129817
Read 129817 sentence pairs
Trimmed to 129817 sentence pairs
Counting words...
Counted words:
kan 53788
eng 19569
['ಆಲಿಸ್\u200cಗೆ ಹಾಗೆ ಮಾಡುವಂತೆ ಕೇಳಲು ಮೇರಿ ಜಾನ್\u200cಗೆ ಹೇಳಿದಳು ಎಂದು ಟಾಮ್\u200cಗೆ ತಿಳಿದಿತ್ತು', 'Tom knew that Mary told John to ask Alice to do that']


In [ ]:
#Defining objects to create our lookup dictionaries for deployment
input_lkp = input_lang_kan.word2index
output_lkp = output_lang_eng.index2word

In [ ]:
len(input_lkp)

19667

In [ ]:
#Checking maximum length of sentence in the given dataset
for i in range(len(pairs)):
  a = 0
  b = 0
  if len(pairs[i][0].split()) > a:
    a = max(len(pairs[i][0].split()),a)
  if len(pairs[i][1].split())>b:
    b = max(len(pairs[i][1].split()),b)
print(a)
print(b)
#Maximum length of the kannada Sentence is 7
#maximum length of a english sentence is 14
#Hence Declaring Max_length as 15
MAX_LENGTH = 15

11
12


In [ ]:
#Defining the class for the Encoder, the construct of layers and the RNN for encoding a given input sentence
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
#Decoder have the layers of nueral network and RNN as defined below, this class would work along with attention mechanism
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
#This class would use attention weights to each hidden state of encoder and build the context vector that would go in decoding the words in a sequence.
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
# We define the below functions to generate tensors from the text using the indexes assigned to each word. These tensors are used to feed the network
def indexesFromSentence(lang, sentence):
    l=[]
    for word in sentence.split(' '):
        ids=lang.word2index[word]
        l.append(ids)
        
            
    return l


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang_kan, pair[0])
    target_tensor = tensorFromSentence(output_lang_eng, pair[1])
    return (input_tensor, target_tensor)

In [ ]:
teacher_forcing_ratio = 0.5
# We define the training function below
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
def evaluate_eng(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang_kan, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang_eng.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()
            sen = ' '.join(decoded_words)

        return sen

In [ ]:
#Training the model
hidden_size = 100
encoder_eng = EncoderRNN(input_lang_kan.n_words, hidden_size).to(device)
attn_decoder_eng = AttnDecoderRNN(hidden_size, output_lang_eng.n_words,dropout_p=0.1).to(device)

trainIters(encoder_eng, attn_decoder_eng,50000, print_every=2000)

1m 3s (- 25m 25s) (2000 4%) 5.8486
2m 5s (- 24m 0s) (4000 8%) 5.9451
3m 8s (- 23m 3s) (6000 12%) 5.9950
4m 11s (- 22m 1s) (8000 16%) 5.8802
5m 14s (- 20m 56s) (10000 20%) 5.7476
6m 17s (- 19m 53s) (12000 24%) 5.7767
7m 20s (- 18m 53s) (14000 28%) 5.6966
8m 24s (- 17m 51s) (16000 32%) 5.5995
9m 27s (- 16m 48s) (18000 36%) 5.6163
10m 30s (- 15m 46s) (20000 40%) 5.5256
11m 34s (- 14m 44s) (22000 44%) 5.5551
12m 38s (- 13m 42s) (24000 48%) 5.4959
13m 43s (- 12m 39s) (26000 52%) 5.3873
14m 47s (- 11m 37s) (28000 56%) 5.3723
15m 51s (- 10m 34s) (30000 60%) 5.3372
16m 56s (- 9m 31s) (32000 64%) 5.3540
17m 59s (- 8m 28s) (34000 68%) 5.2319
19m 4s (- 7m 24s) (36000 72%) 5.2624
20m 7s (- 6m 21s) (38000 76%) 5.1453
21m 11s (- 5m 17s) (40000 80%) 5.1003
22m 16s (- 4m 14s) (42000 84%) 5.2002
23m 19s (- 3m 10s) (44000 88%) 5.0822
24m 23s (- 2m 7s) (46000 92%) 5.0272
25m 27s (- 1m 3s) (48000 96%) 5.0578
26m 31s (- 0m 0s) (50000 100%) 5.0081


In [ ]:

print("complete")

complete


In [ ]:
#Evaluation for comparing the result
evaluate_eng(encoder_eng, attn_decoder_eng,"ನಾನು ತುಂಬಾ ಪ್ರಭಾವಿತನಾಗಿದ್ದೇನೆ")

"I'm not very"

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

ans=0
val=0
ok=0
done=0
print(len(X_test))
itr=0
for i,j in zip(X_train,y_train):
#     print(i,j)
    itr=itr+1
    # if(itr==10):break
    if(len(i.split())>15 or len(j.split())>15 ):continue
    reference = []
    maxa=0
    # print(i)
    reference.append(normalizeString(i).split())
    try:
        candidate = evaluate_eng(encoder_eng, attn_decoder_eng,j)
        
    except:
        # print(ans,"-->",ok)
        continue
    # print(reference,normalizeString(candidate).split(),sentence_bleu(reference, normalizeString(candidate).split() ))
    val=val+sentence_bleu(reference, normalizeString(candidate).split() )
    done=done+1
    ok=max(ok,sentence_bleu(reference, normalizeString(candidate).split() ))
    # print(sentence_bleu(reference, normalizeString(candidate).split() ))
    #     if(ans==2000):break
    
    
print(val/done,ok,ans,done,maxa)
#pm-india
#test-> 0.3774(520)
#train-> 0.37(7529)

#180k dataset
#train---> 0.41(129817)
#test---->0.41(45058)

3227


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.37650987526370866 1.0 0 7529 0
